In [ ]:
def get_flatten_iab_tags(csv_path: str) -> list[str]:
    iab_tags = list(map(str.strip, open(csv_path).readlines()[1:]))  # skip header
    flatten_tags = [
        ': '.join(map(str.strip, filter(bool, line.split(','))))  # split with ', ', strip and join with ': '
        for line in iab_tags
    ]
    return list(filter(bool, flatten_tags))  # filter empty strings


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from torch import nn
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
iab_tags = get_flatten_iab_tags('IAB_tags.csv')
iab_tags[:5]

['Транспорт',
 'Транспорт: Типы кузова автомобиля',
 'Транспорт: Типы кузова автомобиля: Грузовой автомобиль',
 'Транспорт: Типы кузова автомобиля: Седан',
 'Транспорт: Типы кузова автомобиля: Универсал']

In [ ]:
class VideoDataset(Dataset):
   def __init__(self, descriptions, tags, tokenizer, max_len):
	   self.descriptions = descriptions
		self.tags = tags
		self.tokenizer = tokenizer
		self.max_len = max_len
		self.max_tags = len(iab_tags)

	def __len__(self):
		return len(self.descriptions)

	def __getitem__(self, idx):
		description = str(self.descriptions[idx])
		tag = self.tags[idx]

		inputs = self.tokenizer(description, max_length=self.max_len, padding='max_length', truncation=True, return_tensors='pt')

		tag_tensor = np.zeros(self.max_tags)
		for t in tag:
			if t < self.max_tags:
				tag_tensor[t] = 1.0

		return {
			'input_ids': inputs['input_ids'].flatten(),
			'attention_mask': inputs['attention_mask'].flatten(),
			'labels': torch.tensor(tag_tensor, dtype=torch.float)
		}

In [ ]:
class VideoTaggerModel(BertForSequenceClassification):
    def forward(self, input_ids, attention_mask=None, labels=None, **kwargs):
        outputs = super(VideoTaggerModel, self).forward(input_ids, attention_mask=attention_mask, **kwargs)
        logits = outputs.logits

        if labels is not None:
            loss_fct = nn.BCEWithLogitsLoss()
            loss = loss_fct(logits, labels)
            return loss, logits

        return logits

In [ ]:
def split_tags(tags: str):
	splitted_tags = list(filter(bool, map(str.strip, str(tags).replace('  ', ' ').split(','))))
	return splitted_tags


df = pd.read_csv('train_data_categories.csv')
df['splitted_tags'] = df['tags'].apply(split_tags)
df.head()

,video_id,title,description,tags,splitted_tags
0,9007f33c8347924ffa12f922da2a179d,Пацанский клининг. Шоу «ЧистоТачка» | Повелите...,Тяпа и Егор бросили вызов нестареющему «повели...,Массовая культура: Юмор и сатира,[Массовая культура: Юмор и сатира]
1,9012707c45233bd601dead57bc9e2eca,"СarJitsu. 3 сезон, 6 серия. Нарек Симонян vs Ж...","CarJitsu — бои в формате POP MMA, где вместо р...",События и достопримечательности: Спортивные с...,[События и достопримечательности: Спортивные с...
2,e01d6ebabbc27e323fa1b7c581e9b96a,"Злые языки | Выпуск 1, Сезон 1 | Непорочность ...",Почему Дана Борисова предпочитает молчать о по...,"Массовая культура: Отношения знаменитостей, Ма...","[Массовая культура: Отношения знаменитостей, М..."
3,a00b145242be3ebc3b311455e94917af,$1000 шоу | 1 выпуск | Автобоулинг,"В этом выпуске, популярный автоблогер Дима Гор...","Транспорт, Спорт: Автогонки, Массовая культура","[Транспорт, Спорт: Автогонки, Массовая культура]"
4,b01a682bf4dfcc09f1e8fac5bc18785a,В РОТ МНЕ НОТЫ #1 ВИТА ЧИКОВАНИ,В первом выпуске «В рот мне ноты» популярная п...,Массовая культура: Юмор и сатира,[Массовая культура: Юмор и сатира]


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
max_len = 256

tags_encoder = {tag: idx for idx, tag in enumerate(iab_tags)}
df['encoded_tags'] = df['splitted_tags'].apply(
	lambda tags: np.array([tags_encoder.get(tag, 0) for tag in tags])
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
train_df, test_df = train_test_split(df, test_size=0.1)
train_dataset = VideoDataset(train_df['description'].values, train_df['encoded_tags'].tolist(), tokenizer, max_len)
test_dataset = VideoDataset(test_df['description'].values, test_df['encoded_tags'].tolist(), tokenizer, max_len)

In [ ]:
model = VideoTaggerModel.from_pretrained('DeepPavlov/rubert-base-cased', num_labels=len(iab_tags))
for param in model.parameters():
    param.data = param.data.contiguous()

training_args = TrainingArguments(
   output_dir='./results',
   num_train_epochs=3,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   warmup_steps=500,
   weight_decay=0.01,
   logging_dir='./logs'
)

trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=train_dataset,
   eval_dataset=test_dataset
)

Some weights of VideoTaggerModel were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [49]:
trainer.train()

Step,Training Loss


Step,Training Loss


TrainOutput(global_step=177, training_loss=0.5312618104751501, metrics={'train_runtime': 149.6781, 'train_samples_per_second': 18.921, 'train_steps_per_second': 1.183, 'total_flos': 374599083147264.0, 'train_loss': 0.5312618104751501, 'epoch': 3.0})

In [50]:
np_iab_tags = np.array(iab_tags)

In [61]:
def prepare_input(description, tokenizer, max_len):
    inputs = tokenizer(description, max_length=max_len, padding='max_length', truncation=True, return_tensors='pt')
    return inputs['input_ids'].to(device), inputs['attention_mask'].to(device)


def predict_tags(description, model, tokenizer, threshold=0.5, max_len=max_len):
    input_ids, attention_mask = prepare_input(description, tokenizer, max_len=max_len)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        if isinstance(outputs, torch.Tensor):
            logits = outputs
        else:
            logits = outputs.logits

    probabilities = torch.sigmoid(logits)
    predicted_tags = (probabilities > threshold).int()

    return np_iab_tags[predicted_tags.cpu().numpy().flatten() == 1]

In [62]:
predict_tags('Актёрское искусство. Как вести себя на кастинге. Ошибки молодых актёров.', model, tokenizer, threshold=0.43)

array(['Бизнес и финансы: Промышленность и сфера услуг: Энергетическая промышленность',
       'Образование: Высшее образование',
       'Массовая культура: Скандалы знаменитостей'], dtype='<U97')

In [93]:
def expand_tag(tag: str) -> list[str]:
    subtags = tag.split(': ')
    return [': '.join(subtags[:i]) for i in range(1, len(subtags) + 1)]

def count_iou(test_tags, predicted_tags):
    set1, set2 = set(), set()
    for tag in test_tags:
        set1.update(expand_tag(tag))
    for tag in predicted_tags:
        set2.update(expand_tag(tag))
    return float(len(set1 & set2) / len(set1 | set2))

In [74]:
expand_tag('Бизнес и финансы: Промышленность и сфера услуг: Энергетическая промышленность')

['Бизнес и финансы',
 'Бизнес и финансы: Промышленность и сфера услуг',
 'Бизнес и финансы: Промышленность и сфера услуг: Энергетическая промышленность']

In [100]:
thresholds = 40, 45, 50

for th in thresholds:
    test_df[f'predicted_tags_{th}'] = test_df['description'].apply(lambda x: predict_tags(x, model, tokenizer, threshold=th / 100))
    test_df[f'iou_{th}'] = np.array([count_iou(x, y) for x, y in zip(test_df['splitted_tags'].values, test_df[f'predicted_tags_{th}'].values)])

test_df

,video_id,title,description,tags,splitted_tags,encoded_tags,predicted_tags_50,predicted_tags_45,iou_50,iou_45,predicted_tags_40,iou_40
137,e1fd71fc161933ac2bb9ad430f316d4d,МАКСИМ НАРОДНЫЙ Выпуск №13 ГОТОВИМ ЧАШУШУЛИ ПО...,Предлагаю подписчикам приготовить чашушули по-...,Еда и напитки: Кулинария,[Еда и напитки: Кулинария],[202],"[Образование: Высшее образование, Массовая кул...",[Бизнес и финансы: Промышленность и сфера услу...,0.0,0.000000,[Транспорт: Типы автомобилей: Беспилотные авто...,0.000000
701,aabedcb044b0c6b093e313377eeef77d,День с художником | Роман Казус,В данном выпуске мы познакомим вас с творчеств...,Хобби и интересы: Декоративно-прикладное искус...,[Хобби и интересы: Декоративно-прикладное иску...,"[237, 112]","[Образование: Высшее образование, Массовая кул...","[Образование: Высшее образование, Массовая кул...",0.0,0.000000,[Бизнес и финансы: Промышленность и сфера услу...,0.058824
540,4858245da2dd61ddae497f48f66de214,"Такой расклад. Руны подскажут, как наладить ли...","Невероятно, но факт: руны знают всё. И даже ка...",Религия и духовность: астрология,[Религия и духовность: астрология],[0],[],[Бизнес и финансы: Промышленность и сфера услу...,0.0,0.000000,"[Бизнес и финансы: Бизнес: Стартапы, Бизнес и ...",0.000000
547,e85d2af5f1b3b26a4618dda3223e4ef2,Тру ДЕТЕКТОР I #13,Вы смотрите шоу с детектором лжи «Тру ДЕТЕКТОР...,Массовая культура: Юмор и сатира,[Массовая культура: Юмор и сатира],[406],[],[Бизнес и финансы: Промышленность и сфера услу...,0.0,0.125000,[Транспорт: Типы автомобилей: Подержанные авто...,0.076923
93,a13338b2adcbe3dbb441c5f6e0e3d7da,100 главных русских изобретений | Выпуск 6| си...,"Это открытие известно и в России, и в Голливуд...","Массовая культура, Наука","[Массовая культура, Наука]","[398, 429]",[],[Образование: Высшее образование],0.0,0.000000,"[Бизнес и финансы: Бизнес: Стартапы, Бизнес и ...",0.052632
...,...,...,...,...,...,...,...,...,...,...,...,...
916,fdab98598407b22bdf49891fe3c02a60,"TikTok Дайджест I 2 сезон, 5 выпуск I У Гаврил...",Егор Крид спалил нос Амины! А у Юли Гаврилиной...,Массовая культура: Отношения знаменитостей,[Массовая культура: Отношения знаменитостей],[403],[],[Бизнес и финансы: Промышленность и сфера услу...,0.0,0.166667,[Бизнес и финансы: Промышленность и сфера услу...,0.062500
24,305a910846f9d43507d2a2147510d4d7,Иордания. Сколько стоит отдых?,В этом выпуске Сашу Великолепного отправили от...,Путешествия: Направления путешествий: Азия,[Путешествия: Направления путешествий: Азия],[570],[],[Бизнес и финансы: Промышленность и сфера услу...,0.0,0.000000,[Бизнес и финансы: Промышленность и сфера услу...,0.000000
273,7444a2ce14b3a95644df679b775e171a,Антон Протеинов I #30 I Как всегда быть в форме,"Антон Протеинов раскрыл секрет, как быть в фор...",Массовая культура: Юмор и сатира,[Массовая культура: Юмор и сатира],[406],[Образование: Высшее образование],[Бизнес и финансы: Промышленность и сфера услу...,0.0,0.100000,[Бизнес и финансы: Промышленность и сфера услу...,0.052632
228,4354a1ad8bf75f42466420f4b52dcbcd,Артмеханика. Концерт группы Диктофон.,Концерт группы Диктофон.,"Массовая культура, Карьера, События и достопри...","[Массовая культура, Карьера, События и достопр...","[398, 112, 170]",[Образование: Высшее образование],[Бизнес и финансы: Промышленность и сфера услу...,0.0,0.125000,[Бизнес и финансы: Промышленность и сфера услу...,0.100000


In [102]:
test_df.to_csv('test.csv')

In [101]:
[test_df[f'iou_{th}'].mean() for th in thresholds]

[0.044988609184319045, 0.05252776324204895, 0.018412698412698412]